## Fixes the final event-code inconsistencies in the shift condition for Attention Shift

This script cross-checks for consistency in `_eventstemp3.tsv` and creates `_eventstemp4.tsv` with corrected version.

### Checking for forbidden codes
       Codes 1 and 2 can appear anywhere
       Codes 3 through 6 should appear only in the focus condition.
       Codes 7 through 14 should appear only in the shift condition.
       Codes 199, 201, 202, and 255 are not related to condition.

In [1]:
import os
import datetime
from hed.tools import BidsTabularDictionary, get_file_list, get_new_dataframe, HedLogger

# Variables to set for the specific dataset
bids_root_path = '/XXX/AttentionShiftWorking'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'run')
skip_cols = ['onset', 'duration', 'sample']
log_name = 'attention_shift_06_fix_modality_code_inconsistencies_log'

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Make the dictionary for _events_temp3
bids_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_eventstemp3", exclude_dirs=exclude_dirs)
bids_dict = BidsTabularDictionary("Event files", bids_files, entities=entities)

In [2]:
for key, file, rowcount, columns in bids_dict.iter_tsv_info():
    filename = file.file_path
    df_bids = get_new_dataframe(filename)
    shift_cond_mask = df_bids['cond_code'].map(str).isin(['3'])
    visual_shift_mask = shift_cond_mask & df_bids['focus_modality'].map(str).isin(['visual'])
    auditory_shift_mask = shift_cond_mask & df_bids['focus_modality'].map(str).isin(['auditory'])

    bad_7 = visual_shift_mask & df_bids['event_code'].map(str).isin(['7'])
    if sum(bad_7):
        df_bids.loc[df_bids.index[bad_7], 'event_code'] = '9'
        logger.add(key, f"{key} has {sum(bad_7)} event code values of 7 in visual changing to 9")
        logger.add(key, f"{str(list(df_bids.index[bad_7]))}")

    bad_8 = auditory_shift_mask & df_bids['event_code'].map(str).isin(['8'])
    if sum(bad_8):
        df_bids.loc[df_bids.index[bad_8], 'event_code'] = '10'
        logger.add(key, f"{key} has {sum(bad_8)} event code values of 8 in auditory changing to 10")
        logger.add(key, f"{str(list(df_bids.index[bad_8]))}")

    bad_9 = auditory_shift_mask & df_bids['event_code'].map(str).isin(['9'])
    if sum(bad_9):
        df_bids.loc[df_bids.index[bad_9], 'event_code'] = '7'
        logger.add(key, f"{key} has {sum(bad_9)} event code values of 9 in auditory changing to 7")
        logger.add(key, f"{str(list(df_bids.index[bad_9]))}")

    bad_10 = visual_shift_mask & df_bids['event_code'].map(str).isin(['10'])
    if sum(bad_10):
        df_bids.loc[df_bids.index[bad_10], 'event_code'] = '8'
        logger.add(key, f"{key} has {sum(bad_10)} event code values of 10 in visual changing to 8")
        logger.add(key, f"{str(list(df_bids.index[bad_10]))}")

    bad_11 = visual_shift_mask & df_bids['event_code'].map(str).isin(['11'])
    if sum(bad_11):
        df_bids.loc[df_bids.index[bad_11], 'event_code'] = '13'
        logger.add(key, f"{key} has {sum(bad_11)} event code values of 11 in visual changing to 13")
        logger.add(key, f"{str(list(df_bids.index[bad_11]))}")

    bad_12 = auditory_shift_mask & df_bids['event_code'].map(str).isin(['12'])
    if sum(bad_12):
        df_bids.loc[df_bids.index[bad_12], 'event_code'] = '14'
        logger.add(key, f"{key} has {sum(bad_12)} event code values of 12 in visual changing to 14")
        logger.add(key, f"{str(list(df_bids.index[bad_12]))}")

    bad_13 = auditory_shift_mask & df_bids['event_code'].map(str).isin(['13'])
    if sum(bad_13):
        df_bids.loc[df_bids.index[bad_13], 'event_code'] = '11'
        logger.add(key, f"{key} has {sum(bad_13)} event code values of 13 in visual changing to 11")
        logger.add(key, f"{str(list(df_bids.index[bad_13]))}")

    bad_14 = visual_shift_mask & df_bids['event_code'].map(str).isin(['14'])
    if sum(bad_14):
        df_bids.loc[df_bids.index[bad_14], 'event_code'] = '12'
        logger.add(key, f"{key} has {sum(bad_14)} event code values of 14 in visual changing to 12")
        logger.add(key, f"{str(list(df_bids.index[bad_14]))}")
    filename_new = filename[:-5] + "4.tsv"
    df_bids.to_csv(filename_new , sep='\t', index=False)
    logger.add(key, f"Saving {filename_new}")

# Now fix subject 021 run 2 with missing shift code
key = 'sub-021_run-02'
filename = bids_dict.get_file_path(key)
filename_new = filename[:-5] + "4.tsv"
df = get_new_dataframe(filename_new)
df.loc[df.index[576:586], 'focus_modality'] = 'auditory'
logger.add(key, "Updating focus_modality for 576:586 from n/a to auditory")
logger.add(key, f"Saving {filename_new}")
df.to_csv(filename_new, sep='\t', index=False)

# Output and save the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
print(log_string)
print(error_string)

save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)




Log output:
attention_shift_06_fix_modality_code_inconsistencies_log: Level None
sub-001_run-01:
	[ sub-001_run-01 has 1 event code values of 9 in auditory changing to 7]
	[ [2462]]
	[ sub-001_run-01 has 4 event code values of 11 in visual changing to 13]
	[ [883, 884, 1192, 1193]]
	[ sub-001_run-01 has 7 event code values of 12 in visual changing to 14]
	[ [820, 2456, 2458, 2460, 2464, 2465, 2467]]
	[ sub-001_run-01 has 5 event code values of 13 in visual changing to 11]
	[ [2457, 2459, 2461, 2463, 2466]]
	[ Saving G:\AttentionShift\AttentionShiftWorking\sub-001\eeg\sub-001_task-AuditoryVisualShift_run-01_eventstemp4.tsv]
sub-002_run-01:
	[ sub-002_run-01 has 4 event code values of 11 in visual changing to 13]
	[ [880, 881, 1193, 1194]]
	[ sub-002_run-01 has 1 event code values of 12 in visual changing to 14]
	[ [815]]
	[ Saving G:\AttentionShift\AttentionShiftWorking\sub-002\eeg\sub-002_task-AuditoryVisualShift_run-01_eventstemp4.tsv]
sub-003_run-01:
	[ sub-003_run-01 has 4 event c